In [1]:
import numpy as np
import plotly.graph_objects as go
import csv
import scipy as sp
import sys,os

In [2]:
def read_data(file_path):

    with open(file_path) as f:
        reader = csv.reader(f)
        data = []
        for column in reader:
            column[0] = column[0].replace('(', '').replace("'" , '')
            column[2] = column[2].replace(')', '').replace("'" , '')
            column[1] = column[1].replace("'" , '')
            column[0] = float(column[0])
            column[1] = float(column[1])
            column[2] = float(column[2])
            data.append(column)
            print(data)
        data = np.array(data)
    return data
        # print(data)

In [3]:
#Read energy in GeV of data files, save to list named energies
energies = []
#Read all files in data folder
for root, dir, files in os.walk('../data'):
    for file in files:
        if file.endswith('.csv'):
            energies.append(file.replace('.csv', '').replace('_', '.').replace('G', ''))
energies = np.array(energies, dtype=float)

In [4]:
results = []
for root, dir, files in os.walk('../data'):
    for file in files:
        if file.endswith('.csv'):
            print(file)
            data = read_data(os.path.join(root, file))
            results.append(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




[[-27.5, 112.5, -490.0], [-77.5, 82.5, -490.0], [-47.5, -102.5, -490.0], [7.5, -112.5, -490.0], [-52.5, 102.5, -490.0], [-7.5, 112.5, -490.0], [57.5, -102.5, -490.0], [107.5, 47.5, -490.0], [-82.5, -82.5, -490.0], [112.5, 27.5, -490.0], [-52.5, 102.5, -490.0], [-107.5, 37.5, -490.0], [22.5, 112.5, -490.0], [-92.5, 72.5, -490.0], [-92.5, 67.5, -490.0], [82.5, 77.5, -490.0], [-27.5, -112.5, -490.0], [47.5, -107.5, -490.0], [22.5, -112.5, -490.0], [-97.5, -62.5, -490.0], [112.5, -22.5, -490.0], [27.5, -112.5, -490.0], [-107.5, 42.5, -490.0], [-107.5, -37.5, -490.0], [107.5, 42.5, -490.0], [27.5, -112.5, -490.0], [87.5, -77.5, -490.0], [-52.5, 102.5, -490.0], [-77.5, 82.5, -490.0], [-112.5, -7.5, -490.0], [-2.5, 112.5, -490.0], [-87.5, 72.5, -490.0], [47.5, 102.5, -490.0], [-112.5, -27.5, -490.0], [22.5, 112.5, -490.0], [-32.5, 107.5, -490.0], [-77.5, -82.5, -490.0], [-82.5, 77.5, -490.0], [112.5, 22.5, -490.0], [107.5, 37.5, -490.0], [-117.5, 2.5, -490.0], [-92.5, -72.5, -490.0], [67.5, 

In [5]:
for i, array in enumerate(results):
    print(f'data set{i} shape: {array.shape}')

data set0 shape: (24, 3)
data set1 shape: (48, 3)
data set2 shape: (97, 3)
data set3 shape: (135, 3)
data set4 shape: (129, 3)
data set5 shape: (184, 3)
data set6 shape: (390, 3)
data set7 shape: (202, 3)
data set8 shape: (712, 3)
data set9 shape: (552, 3)
data set10 shape: (607, 3)
data set11 shape: (1001, 3)
data set12 shape: (717, 3)
data set13 shape: (730, 3)
data set14 shape: (819, 3)
data set15 shape: (2314, 3)
data set16 shape: (938, 3)
data set17 shape: (489, 3)
data set18 shape: (1942, 3)
data set19 shape: (1041, 3)
data set20 shape: (1098, 3)
data set21 shape: (1077, 3)


In [ ]:
def plot_pos(arrays, title, directory):
    # if not os.path.exists(directory):
    #     os.makedirs(directory)
    for i, array in enumerate(arrays):
        # np.savetxt(f'{directory}/array_{i}.csv', array, delimiter=',')
        # print(f"plotting array {array[:,0]}")
        energy_str = str(energies[i]).replace('.', '_')
        fig = go.Figure(data=go.Scatter(x=array[:,0], y=array[:,1], mode='markers'))
        fig.update_layout(title=f"{energy_str}"+ 'G', xaxis_title='x', yaxis_title='y')
        fig.update_layout(
            width=500,
            height=500,
            margin=dict(
                l=50,
                r=50,
                b=100,
                t=100,
                pad=4
            ),
            xaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            ),
            yaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            )
        )
        # fig.show()
        # fig.write_image('../media/plots/'+title+'.png')
        # fig.write_image("../media/plots/"+f"{energies[i]}"+ 'G'+".png")


        print("../media/plots/"+f"{energy_str}"+ 'G'+".png")

In [ ]:
plot_dir = '../media/plots'
plot_pos(results, 'GeV', plot_dir)

# Calculate emision angle

In [ ]:
def momentum(E, m):
    return np.sqrt(E**2 - m**2)

In [ ]:
def fit_circle(data):
    x, y = data[:,0], data[:,1]
    x_m, y_m = np.mean(x), np.mean(y)

    def calc_R(xc, yc):
        """ calculate the distance of each 2D points from the center (xc, yc) """
        return np.sqrt((x-xc)**2 + (y-yc)**2)

    def f(c):
        """ calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
        Ri = calc_R(*c)
        return Ri - Ri.mean()

    center_estimate = x_m, y_m
    center, _ = sp.optimize.leastsq(f, center_estimate)

    xc, yc = center
    Ri       = calc_R(*center)
    R       = Ri.mean()
    return xc, yc, R

In [ ]:
radii = []
p = []
for array in results:
    part_mom = momentum(energies, 0.139)
    radii.append(fit_circle(array)[2])
    p.append(part_mom)
radii = np.array(radii)
p = np.array(p)
print(radii.shape, energies.shape, p.shape)

processed_data = np.column_stack((energies, radii, p))
print(processed_data.shape)

In [ ]:
def plot_circle(xc, yc, R):
    theta_fit = np.linspace(0, 2*np.pi, 180)
    x_fit = xc + R*np.cos(theta_fit)
    y_fit = yc + R*np.sin(theta_fit)
    fig = go.Figure(data=go.Scatter(x=x_fit, y=y_fit, mode='lines'))
    fig.update_layout(title='0_9G', xaxis_title='y', yaxis_title='z')
    fig.update_layout(
        width=500,
        height=500,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
        xaxis=dict(
            showgrid=True,
            zeroline=True,
            showline=True,
            showticklabels=True,
            scaleratio=1,
            scaleanchor="x",
            range=[-200, 200]
        ),
        yaxis=dict(
            showgrid=True,
            zeroline=True,
            showline=True,
            showticklabels=True,
            scaleratio=1,
            scaleanchor="y",
            range=[-200, 200]
        )
    )
    fig.show()



In [ ]:
yc, zc, Rc = fit_circle(second)
print(f"Center at ({yc}, {zc}) with radius {Rc}")
print(yc, zc, Rc)

In [ ]:
plot_circle(yc, zc, Rc)

In [ ]:
# distancia 0.24 m
def find_angle(d, R ):
    theta = np.arctan(R/d)
    return theta


In [ ]:
angle = find_angle(240, Rc)

In [ ]:
print(angle)

In [ ]:
# plot the cherenkov angle cone
# def plot_cone(data, angle):
#     x, y, z = data[:,0], data[:,1], data[:,2]
#     theta = np.linspace(0, 2*np.pi, 180)
#     # print(theta)
#     x_cone = x + 500*np.cos(theta)
#     y_cone = y + 500*np.sin(theta)
#     z_cone = z + 500*np.tan(angle)*np.ones(180)
#     fig = go.Figure(data=go.Scatter3d(x=x_cone, y=y_cone, z=z_cone, mode='lines'))
#     fig.update_layout(title='0_9G', scene_aspectmode='cube', scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'), width=700, height=700)
#     fig.show()

In [ ]:
# plot_cone(second, angle)